In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
from PIL import Image, ImageOps
import numpy as np

In [2]:

dirpath = pathlib.Path.cwd().joinpath('data/sd/v2_train/x/')
x_data = np.zeros((160, 480, 640, 1))
y_data = np.zeros((160, 81))
for idx, it in enumerate(dirpath.iterdir()):
    outfil = pathlib.Path.cwd().joinpath('data/sd/v2_train/y/').joinpath(it.stem + '.dat')
    # print(outfil)
    f = open(outfil, 'r')
    lns = f.readlines()
    allys = []
    for i, lne in enumerate(lns):
        if i < 2:
            continue
        lne = lne.strip()
        chrs = lne.split(' ')[0:9]
        for c in chrs:
            allys.append(int(c))
    # print(len(allys))
    y = np.array(allys)    
    # print(y)
    y_data[idx] = y.copy()
    img = Image.open(it)
    img = ImageOps.grayscale(img)
    img = img.resize((640, 480))
    arr = np.asarray(img) / 255.0
    arr = arr.reshape(arr.shape + (1,))
    x_data[idx] = arr.copy()
    # print(arr)
print(x_data)
print(y_data)

[[[[0.72156863]
   [0.72156863]
   [0.71764706]
   ...
   [0.07843137]
   [0.07843137]
   [0.07843137]]

  [[0.72156863]
   [0.71764706]
   [0.71764706]
   ...
   [0.07843137]
   [0.07843137]
   [0.07843137]]

  [[0.71372549]
   [0.71372549]
   [0.71372549]
   ...
   [0.0745098 ]
   [0.0745098 ]
   [0.0745098 ]]

  ...

  [[0.68627451]
   [0.68627451]
   [0.68627451]
   ...
   [0.36862745]
   [0.37647059]
   [0.37647059]]

  [[0.68627451]
   [0.68627451]
   [0.68627451]
   ...
   [0.36862745]
   [0.37647059]
   [0.37647059]]

  [[0.68627451]
   [0.68627451]
   [0.68627451]
   ...
   [0.36862745]
   [0.37647059]
   [0.37647059]]]


 [[[0.56078431]
   [0.56862745]
   [0.57647059]
   ...
   [0.6627451 ]
   [0.63921569]
   [0.61960784]]

  [[0.55686275]
   [0.56470588]
   [0.57254902]
   ...
   [0.67058824]
   [0.65098039]
   [0.63137255]]

  [[0.54901961]
   [0.56078431]
   [0.56862745]
   ...
   [0.68235294]
   [0.66666667]
   [0.65490196]]

  ...

  [[0.44313725]
   [0.44313725]
   [0.4

In [3]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same",input_shape=(480, 640, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 480, 640, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 240, 320, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 240, 320, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 120, 160, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 120, 160, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 60, 80, 64)        0

In [5]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(81, activation="relu"))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 480, 640, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 240, 320, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 240, 320, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 120, 160, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 120, 160, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 60, 80, 64)        0

In [7]:
print(y_data.shape)
print(x_data.shape)

(160, 81)
(160, 480, 640, 1)


In [10]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

history = model.fit(x_data, y_data, epochs=10)

Epoch 1/10
5/5 [==============================] - 17s 3s/step - loss: 1382.9749 - accuracy: 0.0063
Epoch 2/10
5/5 [==============================] - 16s 3s/step - loss: 1295.8972 - accuracy: 0.0000e+00
Epoch 3/10
5/5 [==============================] - 17s 3s/step - loss: 1281.9524 - accuracy: 0.0000e+00
Epoch 4/10
5/5 [==============================] - 17s 3s/step - loss: 1281.2175 - accuracy: 0.0000e+00
Epoch 5/10
5/5 [==============================] - 17s 3s/step - loss: 1281.2728 - accuracy: 0.0000e+00
Epoch 6/10
5/5 [==============================] - 17s 3s/step - loss: 1281.3010 - accuracy: 0.0000e+00
Epoch 7/10
5/5 [==============================] - 18s 4s/step - loss: 1281.1947 - accuracy: 0.0000e+00
Epoch 8/10
5/5 [==============================] - 17s 3s/step - loss: 1281.1870 - accuracy: 0.0000e+00
Epoch 9/10
5/5 [==============================] - 17s 3s/step - loss: 1281.0934 - accuracy: 0.0000e+00
Epoch 10/10
5/5 [==============================] - 17s 3s/step - loss: 1280.9